In [1]:
import re
import numpy as np

In [2]:
from TwitterClient import TwitterClient as tc

In [3]:
api = tc()

In [4]:
api.createDataset('ronaldo.txt', '#ronaldo', 20000)

# Read Data

In [5]:
f = open('./Datasets/ronaldo.txt', 'r')
tweets = f.read().split('_$_')
f.close()

# Preprocess

In [6]:
from PreProcessor import PreProcessor as pp

In [7]:
tweets

['RT @_iambatman10: Waiting for Sir Alex. @Cristiano. Together again ❤ \nBE LIKE \n#MUFC #Ronaldo #ggmu #siralex https://t.co/ExQU8KcZhK',
 '#UEFA-CONMEBOL Nations League? \nAll 10 South American nations are planned to join the UEFA Nations League when the… https://t.co/nJJkkWIcAm',
 '#Lewandowski equals Ronaldo’s record of 69 goals in a year with fewer games 🔥🔥🔥\n#978bet #football #ManUnited… https://t.co/6NvDyV0JcW',
 'RT @FutboLPress_: Erling Haaland empató el histórico récord de Ronaldo, ocurrido en 1997, al alcanzar los 48 goles anotados en un año natur…',
 'RT @VikasZarooriHai: The Sir Alex-ronaldo Interview!\n\nhttps://t.co/e1fmhg2ePu\n\n#MUFC #ManUtd Cristiano Ronaldo #SirAlex #MANCHESTER #Manche…',
 'The Sir Alex-ronaldo Interview!\n\nhttps://t.co/e1fmhg2ePu\n\n#MUFC #ManUtd Cristiano Ronaldo #SirAlex #MANCHESTER… https://t.co/rGGkeORpDo',
 '‼MESSI LÀ MỘT ĐẲNG CẤP KHÁC\n#Lewandowski  với phong độ tuyệt vời năm nay vừa cân bằng kỉ lục 69 bàn/1năm dương lịch… https://t.co/ewqJhB

In [8]:
p = pp(tweets)

In [9]:
processed_tweets = p.preProcess()

In [10]:
len(processed_tweets)

63

In [11]:
from PreProcessor import CreateFreqs as cf

In [12]:
c = cf(processed_tweets)

In [13]:
freqs = c.createFreqs()

In [14]:
freqs

{'wait': 2,
 'sir': 7,
 'alex': 5,
 '.  .': 1,
 'togeth': 1,
 'like': 1,
 'mufc': 9,
 'ronaldo': 46,
 'ggmu': 2,
 'siralex': 4,
 'uefa-conmebol': 1,
 'nation': 5,
 'leagu': 5,
 '10': 6,
 'south': 3,
 'american': 3,
 'plan': 2,
 'join': 1,
 'uefa': 1,
 'lewandowski': 6,
 'equal': 3,
 'record': 3,
 '69': 6,
 'goal': 6,
 'year': 4,
 'fewer': 1,
 'game': 1,
 '978bet': 1,
 'footbal': 11,
 'manunit': 1,
 'erl': 2,
 'haaland': 2,
 'empató': 2,
 'el': 2,
 'histórico': 2,
 'récord': 2,
 'de': 3,
 'ocurrido': 2,
 'en': 6,
 '1997': 2,
 'al': 2,
 'alcanzar': 2,
 'lo': 2,
 '48': 2,
 'gole': 2,
 'anotado': 2,
 'un': 3,
 'año': 2,
 'natur': 1,
 'alex-ronaldo': 2,
 'interview': 4,
 'manutd': 4,
 'cristiano': 12,
 'manchest': 4,
 'manch': 1,
 'messi': 14,
 'là': 1,
 'khác': 1,
 'phong': 1,
 'nay': 1,
 'cân': 1,
 'bàn': 1,
 'touch': 1,
 'reach': 1,
 'leomessi': 1,
 'german': 1,
 'cita': 2,
 'darwin': 1,
 'frase': 1,
 'verdad': 1,
 'ego': 1,
 'meme': 1,
 'gent': 1,
 'futbol': 3,
 'reflexion': 1,
 'realid

In [15]:
freq_sorted = c.createFreqSorted()
freq_sorted

[('ronaldo', 46),
 ('messi', 14),
 ('cristiano', 12),
 ('footbal', 11),
 ('deposit', 10),
 ('bonu', 10),
 ('mufc', 9),
 ('sir', 7),
 ('10', 6),
 ('lewandowski', 6),
 ('69', 6),
 ('goal', 6),
 ('en', 6),
 ('bisa', 6),
 ('alex', 5),
 ('nation', 5),
 ('leagu', 5),
 ('pulsa', 5),
 ('tanpa', 5),
 ('potongann', 5),
 ('setiap', 5),
 ('harinya', 5),
 ('untuk', 5),
 ('info', 5),
 ('dan', 5),
 ('menarik', 5),
 ('lainnya', 5),
 ('hubungi', 5),
 ('still', 5),
 ('cristianoronaldo', 5),
 ('cr7', 5),
 ('video', 5),
 ('ferguson', 5),
 ('siralex', 4),
 ('year', 4),
 ('interview', 4),
 ('manutd', 4),
 ('manchest', 4),
 ('team', 4),
 ('app', 4),
 ('creat', 4),
 ('south', 3),
 ('american', 3),
 ('equal', 3),
 ('record', 3),
 ('de', 3),
 ('un', 3),
 ('futbol', 3),
 ('manchesterunit', 3),
 ('realmadrid', 3),
 ('robert', 3),
 ('link', 3),
 ('oldtrafford', 3),
 ('premierleagu', 3),
 ('que', 3),
 ('se', 3),
 ('unit', 3),
 ('footballreel', 3),
 ('realli', 3),
 ('man', 3),
 ('wait', 2),
 ('ggmu', 2),
 ('plan', 2

# Clustering

In [16]:
from PreProcessor import Vectorizer

In [17]:
vectors = Vectorizer(processed_tweets, freq_sorted).textVectorize()

In [18]:
from clustering import KMeans

In [19]:
alphabet = [i[0] for i in freq_sorted]

In [20]:
kmeans = KMeans(4, 100, vectors, alphabet)

In [21]:
clusters = kmeans.run()

In [22]:
clusters

{0: [2, 4, 5, 18, 19, 20, 27, 29, 35, 36, 37, 55, 58, 60],
 1: [7, 16, 24, 42, 49, 50, 51],
 2: [0,
  1,
  3,
  6,
  8,
  9,
  10,
  17,
  21,
  22,
  23,
  25,
  26,
  28,
  30,
  31,
  32,
  33,
  34,
  38,
  39,
  40,
  41,
  43,
  44,
  45,
  46,
  47,
  48,
  52,
  53,
  54,
  56,
  57,
  59,
  61,
  62],
 3: [11, 12, 13, 14, 15]}

In [23]:
print(len(alphabet))

316


In [24]:
processed_tweets[16]

['one',
 'last',
 'danc',
 'messi',
 'vs',
 'ronaldo',
 'messi',
 'ronaldo',
 'uefachampionsleagu',
 'manchesterunit',
 'psg',
 'barcelona',
 'realmadrid',
 'h']

In [25]:
for key in clusters:
    clusters[key] = np.array(clusters[key])

In [26]:
clusters

{0: array([ 2,  4,  5, 18, 19, 20, 27, 29, 35, 36, 37, 55, 58, 60]),
 1: array([ 7, 16, 24, 42, 49, 50, 51]),
 2: array([ 0,  1,  3,  6,  8,  9, 10, 17, 21, 22, 23, 25, 26, 28, 30, 31, 32,
        33, 34, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 52, 53, 54, 56, 57,
        59, 61, 62]),
 3: array([11, 12, 13, 14, 15])}

In [27]:
len(processed_tweets)

63